In [13]:
from glob import glob
import json
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm_notebook
from skimage.io import imread, imsave
from model.retinanet import RetinaNet
from utils import get_anchors, decode_targets, draw_bboxes, get_image

print('TensorFlow', tf.__version__)

TensorFlow 2.0.0-rc0


In [5]:
train_image_paths = sorted(
    glob('../../bdd/bdd100k/images/100k/train/*'))
train_label_paths = sorted(
    glob('../../bdd/bdd100k/labels/100k/train/*'))
validation_image_paths = sorted(
    glob('../../bdd/bdd100k/images/100k/val/*'))
validation_label_paths = sorted(
    glob('../../bdd/bdd100k/labels/100k/val/*'))

print('Found training {} images'.format(len(train_image_paths)))
print('Found training {} labels'.format(len(train_label_paths)))
print('Found validation {} images'.format(len(validation_image_paths)))
print('Found validation {} labels'.format(len(validation_label_paths)))

class_map = {value: idx for idx, value in enumerate(['bus',
                                                     'traffic light',
                                                     'traffic sign',
                                                     'person',
                                                     'bike',
                                                     'truck',
                                                     'motor',
                                                     'car',
                                                     'train',
                                                     'rider'])}
for image, label in zip(train_image_paths, train_label_paths):
    assert image.split(
        '/')[-1].split('.')[0] == label.split('/')[-1].split('.')[0]
for image, label in zip(validation_image_paths, validation_label_paths):
    assert image.split(
        '/')[-1].split('.')[0] == label.split('/')[-1].split('.')[0]

Found training 70000 images
Found training 70000 labels
Found validation 10000 images
Found validation 10000 labels


In [79]:
# t = tf.Variable(0)
t.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=5>

In [7]:
n_classes = len(class_map)
input_shape = 512
BATCH_SIZE = 64
EPOCHS = 2
training_steps = len(train_image_paths) // BATCH_SIZE
validation_steps = len(validation_image_paths) // BATCH_SIZE

model = RetinaNet(input_shape=input_shape, n_classes=n_classes)
model.build([None, input_shape, input_shape, 3])
optimizer = tf.keras.optimizers.Adam(1e-4)
model_dir = 'model_files/'
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint_manager = tf.train.CheckpointManager(checkpoint,
                                                directory=model_dir,
                                                max_to_keep=3)

In [58]:
status = checkpoint.restore('model_files/ckpt-9')

In [67]:
image = get_image(input_shape=512, image_path=train_image_paths[110])[None, ...]
c_preds, r_preds = model(image)
b, c, s, _ = decode_targets(c_preds[0], r_preds[0], classification_threshold=.2, nms_threshold=0.5)
img = (image[0] + tf.constant([103.939, 116.779, 123.68]))[:, :, ::-1]
img = draw_bboxes(img, b).numpy()
imsave('inference.png', img)
b, c

(<tf.Tensor: id=40748, shape=(2, 4), dtype=int32, numpy=
 array([[  9, 200, 141, 364],
        [  1, 218,  88, 378]], dtype=int32)>,
 <tf.Tensor: id=40743, shape=(2,), dtype=int64, numpy=array([7, 7])>)

In [57]:
checkpoint_manager.latest_checkpoint

'model_files/ckpt-8'